# Deepsurv


In [2]:
!pip install -U sentence-transformers > /dev/null 2>&1
!pip install xgboost > /dev/null 2>&1
!pip install scikit-learn==1.4.2 scikit-survival==0.23.1 > /dev/null 2>&1
!pip install torchtuples > /dev/null 2>&1
!pip install pycox > /dev/null 2>&1
!pip install numpy==1.21.5  > /dev/null 2>&1
!pip install interpret-core  > /dev/null 2>&1
!pip install lightgbm > /dev/null 2>&1
!pip install shap > /dev/null 2>&1
!pip install lifelines pycox > /dev/null 2>&1
!pip install pycountry > /dev/null 2>&1
!pip install -U sentence-transformers xgboost scikit-learn==1.4.2 scikit-survival==0.23.1 torchtuples pycox numpy==1.21.5 interpret-core lightgbm shap lifelines pycox pycountry > /dev/null 2>&1


In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torchtuples as tt
import kagglehub
import os

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from lifelines import CoxPHFitter
from sksurv.metrics import concordance_index_censored
from pycox.models.cox import CoxPH
from pycox.evaluation import EvalSurv
from pycox import models
import torch
import torch.nn as nn
from torch.optim import Adam


In [38]:
# --- Pandas Display Options to try to force full output ---
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', 50) # Adjust as needed
pd.set_option('display.width', 1000)     # Adjust as needed
pd.set_option('display.max_colwidth', None) # Show full column content
pd.set_option('display.float_format', '{:.4f}'.format) # Optional: format floats

feature_list = ['stress_score', 'avg_bmi', 'smoking_prev', 'global_life_exp'] # Define feature_list globally


In [4]:
# Life Expectancy dataset
life_exp_path = kagglehub.dataset_download("kumarajarshi/life-expectancy-who")
life_exp_file = os.path.join(life_exp_path, "Life Expectancy Data.csv")
life_exp_df = pd.read_csv(life_exp_file)
print("Life Expectancy Sample:")
print(life_exp_df.head())

# Heart Failure dataset (not used in LightGBM, but kept for context)
heart_path = kagglehub.dataset_download("fedesoriano/heart-failure-prediction")
heart_file = os.path.join(heart_path, "heart.csv")
heart_df = pd.read_csv(heart_file)
print("Heart Failure Sample:")
print(heart_df.head())

# Age Dataset
age_path = kagglehub.dataset_download("imoore/age-dataset")
age_file = os.path.join(age_path, "AgeDataset-V1.csv")
age_df = pd.read_csv(age_file)
print("Age Dataset Sample:")
print(age_df.head())

# World important events Dataset
events_path = kagglehub.dataset_download("saketk511/world-important-events-ancient-to-modern")
events_file = os.path.join(events_path, "World Important Dates.csv")
events_df = pd.read_csv(events_file)
print("World Important Events Sample:")
print(events_df.head())

# Plane Crash Dataset
plane_crash_path = kagglehub.dataset_download("nguyenhoc/plane-crash")
plane_crash_file = os.path.join(plane_crash_path, "planecrashinfo_20181121001952.csv")  
planes_df = pd.read_csv(plane_crash_file)
print("Historical Plane Crashes Sample:")
print(planes_df.head())

# Gloabl Life Expectancy dataset
global_le_path = kagglehub.dataset_download("hasibalmuzdadid/global-life-expectancy-historical-dataset")
global_le_file = os.path.join(global_le_path, "global life expectancy dataset.csv")
global_le_df = pd.read_csv(global_le_file)
print("Global Life Expectancy Historical Dataset Sample:")
print(global_le_df.head())

# US death rate Dataset
death_rates_path = kagglehub.dataset_download("melissamonfared/death-rates-united-states")
death_rates_file = os.path.join(death_rates_path, "Death_rates.csv")
death_rates_df = pd.read_csv(death_rates_file)
print("Death Rates United States Dataset Sample:")
print(death_rates_df.head())

Life Expectancy Sample:
       Country  Year      Status  Life expectancy   Adult Mortality  \
0  Afghanistan  2015  Developing              65.0            263.0   
1  Afghanistan  2014  Developing              59.9            271.0   
2  Afghanistan  2013  Developing              59.9            268.0   
3  Afghanistan  2012  Developing              59.5            272.0   
4  Afghanistan  2011  Developing              59.2            275.0   

   infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   ...  \
0             62     0.01               71.279624         65.0      1154  ...   
1             64     0.01               73.523582         62.0       492  ...   
2             66     0.01               73.219243         64.0       430  ...   
3             69     0.01               78.184215         67.0      2787  ...   
4             71     0.01                7.097109         68.0      3013  ...   

   Polio  Total expenditure  Diphtheria    HIV/AIDS         GD

In [39]:
def enhanced_feature_engineering(df, life_exp_df, global_le_df, death_rates_df):
    """
    Enhanced feature engineering with comprehensive NaN debugging prints.
    """
    print("\n--- Enhanced Feature Engineering ---")

    # -------- Validate Input Columns --------
    required_columns = {'Country', 'Gender', 'Occupation', 'Birth year',
                        'Death year', 'Age of death'}
    missing = required_columns - set(df.columns)
    if missing:
        raise KeyError(f"Missing required columns: {missing}")
    print("✅ Input columns validated.")

    # -------- Set Observation Year --------
    current_year = 2019

    # -------- Handle Initial NaNs in 'Country' --------
    print("\n🔍 Checking and handling initial NaNs in 'Country'...")
    initial_nan_count_country = df['Country'].isnull().sum()
    print(f"   Initial NaNs in Country: {initial_nan_count_country}")

    # Impute NaNs in 'Country' with 'Unknown Country' *before* cleaning
    df['Country'] = df['Country'].fillna('Unknown Country')
    print("   ✅ NaNs in 'Country' imputed with 'Unknown Country'.")
    nan_count_after_imputation = df['Country'].isnull().sum()
    print(f"   NaNs in Country after imputation: {nan_count_after_imputation} (Should be 0)")


    # -------- Basic Cleaning --------
    print("\n🧹 Basic Cleaning...")
    df['Country'] = df['Country'].str.split(';').str[0].str.strip()
    df['Gender'] = np.where(df['Gender'] == 'Male', 1,
                          np.where(df['Gender'] == 'Female', 0, 0.5))
    print("   ✅ Country and Gender cleaned.")
    print(f"   Sample age_df countries after cleaning: {df['Country'].unique()[:20]}")
    print(f"   Sample global_le_df countries: {global_le_df['Country Name'].unique()[:20]}")
    print("\n🔍 Sample global_le_agg countries BEFORE merge:")  # Inspect global_le_agg countries before merge
    global_le_melted = global_le_df.melt(
        id_vars=['Country Name', 'Country Code'],
        value_vars=[str(y) for y in range(1960, current_year+1)],
        var_name='Year',
        value_name='Life_Exp_Value'
    )

    global_le_agg = (
        global_le_melted
        .sort_values(['Country Name', 'Year'], ascending=[True, False])
        .groupby('Country Name')
        ['Life_Exp_Value']
        .first()
        .reset_index()
        .rename(columns={'Country Name': 'Country'})
    )
    print(f"   Unique countries in global_le_agg: {global_le_agg['Country'].unique()[:20]}") # Sample of unique countries in global_le_agg


    # Check for NaNs after basic cleaning (should be minimal now)
    print("🔍 Checking for NaNs after basic cleaning:")
    print(f"   NaNs in Country: {df['Country'].isnull().sum()}") # Expecting 0 or very few
    print(f"   NaNs in Gender: {df['Gender'].isnull().sum()}")


    # -------- Clinical Features (Rest remains the same) --------
    print("\n🧬 Engineering Clinical Features...")
    stress_map = {'Politician': 9, 'Military personnel': 8, 'Journalist': 7,
                  'Businessperson': 6, 'Artist': 5, 'Teacher': 4,
                  'Researcher': 3, 'Other': 5, 'Unknown': 5}
    df['stress_score'] = df['Occupation'].map(stress_map).fillna(5).astype('float32') / 9.0
    print("   ✅ Stress score engineered.")

    life_exp_df[' BMI '] = pd.to_numeric(life_exp_df[' BMI '], errors='coerce')
    country_bmi = life_exp_df.groupby('Country')[' BMI '].median().to_dict()
    df['avg_bmi'] = df['Country'].map(country_bmi).fillna(25).astype('float32')
    print("   ✅ Avg BMI engineered.")

    df['smoking_prev'] = (1 / (1 + np.exp((df['Birth year'] - 1950) / 10))).astype('float32')
    df['smoking_prev'] = np.clip(df['smoking_prev'], 0.1, 0.6)
    print("   ✅ Smoking prevalence engineered.")

    # Check for NaNs after clinical feature engineering
    print("🔍 Checking for NaNs after clinical features:")
    print(f"   NaNs in stress_score: {df['stress_score'].isnull().sum()}")
    print(f"   NaNs in avg_bmi: {df['avg_bmi'].isnull().sum()}")
    print(f"   NaNs in smoking_prev: {df['smoking_prev'].isnull().sum()}")


    # -------- Country-Level Features --------
    print("\n🌍 Engineering Country-Level Features...")
    print("\nSample global_le_agg before merge:") # Inspect global_le_agg
    global_le_melted = global_le_df.melt(
        id_vars=['Country Name', 'Country Code'],
        value_vars=[str(y) for y in range(1960, current_year+1)],
        var_name='Year',
        value_name='Life_Exp_Value'
    )

    global_le_agg = (
        global_le_melted
        .sort_values(['Country Name', 'Year'], ascending=[True, False])
        .groupby('Country Name')
        ['Life_Exp_Value']
        .first()
        .reset_index()
        .rename(columns={'Country Name': 'Country'})
    )
    print(global_le_agg.head()) # Print head of global_le_agg
    print(f"   NaNs in global_le_agg['Country']: {global_le_agg['Country'].isnull().sum()}") # Check NaNs in global_le_agg['Country']
    print(f"   NaNs in global_le_agg['Life_Exp_Value']: {global_le_agg['Life_Exp_Value'].isnull().sum()}") # Check NaNs in global_le_agg['Life_Exp_Value']


    df = df.merge(global_le_agg, on='Country', how='left')
    df['global_life_exp'] = df['Life_Exp_Value'].fillna(df['Life_Exp_Value'].median()) #median imputation to avoid NaN
    print("   ✅ Global Life Expectancy engineered.")

    # Check for NaNs after country-level feature engineering
    print("🔍 Checking for NaNs after country-level features:")
    print(f"   NaNs in global_life_exp: {df['global_life_exp'].isnull().sum()}")


    # -------- Survival Data Setup --------
    print("\n⏳ Survival Data Setup...")
    current_year = 2019 # Re-declare to ensure it's available in this scope if needed

    df['censored'] = (df['Death year'] > current_year).astype(int)

    # Revised 'T' calculation to handle missing 'Age of death' robustly
    df['T'] = np.where(
        df['censored'] == 1, # If censored (still alive as of current_year)
        current_year - df['Birth year'], # T = time until observation (censoring time)
        df['Age of death'] # Else (event observed), T = Age of death
    )

    # Handle potential NaNs in 'T' *after* the calculation, specifically for cases where Age of death is missing for non-censored cases.
    # In such cases, we might default to a large value or impute based on other information if available, or drop the row if no imputation is reasonable.
    # For now, let's impute with a plausible value (e.g., median age of death for non-censored individuals) or simply drop the row for simplicity.
    median_age_death_non_censored = df[df['censored'] == 0]['Age of death'].median()
    df['T'] = df['T'].fillna(median_age_death_non_censored).clip(0, 120) # Impute and clip

    print("   ✅ Survival time (T) and censoring engineered.")

    # Final check for NaNs in critical columns
    print("\n🔍 Final NaN check before returning:")
    print(f"   NaNs in censored: {df['censored'].isnull().sum()}")
    print(f"   NaNs in T: {df['T'].isnull().sum()}")
    print("\nSample processed_batch after feature engineering:") # Inspect processed_batch
    print(df.head())
    print(f"   NaNs in processed_batch after FE: {df[feature_list + ['T', 'censored']].isnull().sum().sum()}") # Total NaNs in features and targets after FE


    print("✅ Enhanced feature engineering completed.\n")
    return df

In [26]:
class DeepSurv(nn.Module):
    def __init__(self, input_dim):
        super(DeepSurv, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

In [44]:
def train_deepsurv_model(df):
    """
    Train and evaluate DeepSurv model with NaN debugging, focusing on 'T'.
    """
    print("\n--- Train DeepSurv Model ---")

    feature_list = ['stress_score', 'avg_bmi', 'smoking_prev', 'global_life_exp']

    # Ensure all required features exist
    missing_features = set(feature_list) - set(df.columns)
    if missing_features:
        raise KeyError(f"Missing required features: {missing_features}. Available: {df.columns.tolist()}")
    print("✅ Required features validated.")

    # Extract Data
    X = df[feature_list].values.astype('float32')
    durations = df['T'].values.astype('float32')
    events = df['censored'].values.astype('int')  # Ensure events are 0 or 1

    # Debugging: Check Data Before Training - NaN Check!
    print("\n📊 Checking Data Before Training DeepSurv (Detailed NaN Check):")
    print(f"Feature Shape: {X.shape}, Durations Shape: {durations.shape}, Events Shape: {events.shape}")
    print("NaN counts per column BEFORE training:")
    nan_counts = df[feature_list + ['T', 'censored']].isnull().sum() # NaN count for each feature and target
    print(nan_counts) # Print NaN counts per column
    print(f"Total NaNs in Features and Targets BEFORE training: {nan_counts.sum()}") # Total NaNs in features and targets
    print(f"Total NaNs in Features (X): {np.isnan(X).sum()}")
    print(f"NaNs in Durations: {np.isnan(durations).sum()}")
    print(f"NaNs in Events: {np.isnan(events).sum()}")
    print(f"Sample Features:\n{X[:5]}")
    print(f"Sample Durations: {durations[:5]}")
    print(f"Sample Events: {events[:5]}")
    print(f"Unique event values (should be 0 or 1): {np.unique(events)}")

    # --- DEBUGGING: FIND ROW WITH NaN in 'T' ---
    nan_T_mask = df['T'].isnull()
    if nan_T_mask.any():
        print("\n🚨🚨🚨 NaN DETECTED in 'T' column! 🚨🚨🚨")
        nan_T_row_index = df[nan_T_mask].index[0] # Get index of first NaN row
        print(f"Index of row with NaN in 'T': {nan_T_row_index}")
        print("Problematic row data:")
        print(df.loc[[nan_T_row_index]]) # Print the entire row
    else:
        print("\n✅ No NaNs detected in 'T' column (so far in NaN check).")


    if np.isnan(X).any() or np.isnan(durations).any() or np.isnan(events).any():
        raise ValueError("NaNs detected in input data, cannot proceed with training.")
    print("✅ No NaNs in input data for training.")


    # Train-Test Split (Rest of the function remains unchanged)
    X_train, X_val, durations_train, durations_val, events_train, events_val = train_test_split(
        X, durations, events, test_size=0.2, random_state=42
    )
    print("   ✅ Data split into training and validation sets.")


    # Normalize Features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    print("   ✅ Features normalized using StandardScaler.")
    print(f"NaNs in Scaled X_train: {np.isnan(X_train).sum()}")
    print(f"NaNs in Scaled X_val: {np.isnan(X_val).sum()}")
    if np.isnan(X_train).any() or np.isnan(X_val).any():
        raise ValueError("NaNs detected in scaled data, check feature engineering and scaling.")
    print("✅ No NaNs in scaled data.")


    # DeepSurv Model Setup
    in_features = X_train.shape[1]
    num_nodes = [64, 64]  # Two hidden layers
    out_features = 1
    batch_norm = True
    dropout = 0.1

    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
    model = models.CoxPH(net, tt.optim.Adam)

    print("\n🚀 Training DeepSurv Model...")
    print(f"✅ DeepSurv Model Initialized with {in_features} input features.")
    print(f"Hidden Layers: {num_nodes}, Dropout: {dropout}")

# ✅ Fix: Correct Loss Logging and Exception Handling
    class LossLogger(tt.callbacks.Callback):
        def on_epoch_end(self):
            if hasattr(self.model, 'log') and self.model.log:
                try:
                    log_entry = self.model.log[-1] # Get the last log entry

                    if isinstance(log_entry, dict): # Check if it's a dictionary as we expect
                        loss = log_entry.get('loss') # Try to get 'loss' from dictionary
                    elif isinstance(log_entry, float) or isinstance(log_entry, int): # Or maybe it's just the loss value directly?
                        loss = log_entry
                    else: # If it's neither, log the type for debugging
                        loss = None
                        print(f"⚠️ Unexpected log entry type: {type(log_entry)}, log entry: {log_entry}")


                    if loss is not None:
                        print(f"Epoch {len(self.model.log)}, Loss: {loss:.5f}")
                    else:
                        print(f"Epoch {len(self.model.log)}, Loss not found in log entry (type: {type(log_entry)}): {log_entry}") # More info
                except Exception as e:
                    print(f"⚠️ Loss logging failed with error: {e}, continuing training... (Error details: {e})") # More detailed error


    # Train Model
    callbacks = [LossLogger()]
    epochs = 100 # Reduced for testing, increase later if needed
    batch_size = 256
    verbose = True # Keep verbose for detailed output

    print(f"Training parameters: Epochs={epochs}, Batch Size={batch_size}, Verbose={verbose}")

    model.fit(
        X_train, (durations_train, events_train),
        batch_size=batch_size, epochs=epochs, verbose=verbose,
        val_data=(X_val, (durations_val, events_val)),
        callbacks=callbacks,
        num_workers=0 # set num_workers to 0 for debugging purposes
    )
    print("✅ DeepSurv model training completed.")


    # Compute Baseline Hazards
    print("\n📊 Computing Baseline Hazards...")
    model.compute_baseline_hazards()
    print("✅ Baseline hazards computed.")


    # Get Predictions
    print("\n🔮 Making Survival Predictions...")
    surv = model.predict_surv_df(X_val)


    # Check for NaNs in Survival Predictions AGAIN after training
    if np.isnan(surv.values).any():
        print("\n❌ WARNING: Survival Predictions STILL Contain NaN Values AFTER TRAINING!")
        nan_count_in_surv = np.isnan(surv.values).sum()
        print(f"   Number of NaNs in Survival Predictions: {nan_count_in_surv}")
    else:
        print("\n✅ No NaN values in survival predictions.")

    # Evaluate Model
    print("\n📈 Evaluating Model Performance...")
    ev = EvalSurv(surv, durations_val, events_val, censor_surv='km')
    c_index = ev.concordance_td('antolini')

    print(f"\n📊 DeepSurv Concordance Index: {c_index:.4f}")

    print("✅ DeepSurv model training and evaluation completed.\n")
    return model, X_val, durations_val, events_val


In [35]:
def evaluate_deepsurv_model(model, X_val, durations_val, events_val):
    """
    Evaluate DeepSurv Model Performance using Concordance Index
    """
    surv = model.predict_surv_df(X_val)
    surv.index = pd.to_numeric(surv.index, errors='coerce')

    if 0 not in surv.index:
        new_row = pd.DataFrame(np.ones((1, surv.shape[1])), index=[0], columns=surv.columns)
        surv = pd.concat([new_row, surv])
        surv = surv.sort_index()

    ev = EvalSurv(surv, durations_val, events_val, censor_surv='km')
    c_index = ev.concordance_td('antolini')

    print(f"\n📊 DeepSurv Concordance Index: {c_index:.4f}")
    return c_index

In [45]:
# 1️⃣ Process the Dataset
processed_batch = enhanced_feature_engineering(age_df, life_exp_df, global_le_df, death_rates_df)
print(f"   Sample age_df countries: {processed_batch['Country'].unique()[:20]}") # Print first 20 unique countries

# 2️⃣ Train DeepSurv
deepsurv_model, X_val, durations_val, events_val = train_deepsurv_model(processed_batch)

# 3️⃣ Evaluate Model
evaluate_deepsurv_model(deepsurv_model, X_val, durations_val, events_val)



--- Enhanced Feature Engineering ---
✅ Input columns validated.

🔍 Checking and handling initial NaNs in 'Country'...
   Initial NaNs in Country: 0
   ✅ NaNs in 'Country' imputed with 'Unknown Country'.
   NaNs in Country after imputation: 0 (Should be 0)

🧹 Basic Cleaning...
   ✅ Country and Gender cleaned.
   Sample age_df countries after cleaning: ['United States of America' 'United Kingdom' 'Archduchy of Austria'
 'Holy Roman Empire' 'Kingdom of France' 'France' 'Spain'
 'Grand Duchy of Tuscany' 'Chile' 'Nazi Germany' 'Kingdom of Castile'
 'Kingdom of the Netherlands' 'Byelorussian Soviet Socialist Republic'
 'Czech Republic' 'Jamaica' 'German Empire' 'Denmark' 'Soviet Union'
 'French Third Republic' 'Unknown Country']
   Sample global_le_df countries: ['Aruba' 'Afghanistan' 'Angola' 'Albania' 'United Arab Emirates'
 'Argentina' 'Armenia' 'Antigua and Barbuda' 'Australia' 'Austria'
 'Azerbaijan' 'Burundi' 'Belgium' 'Benin' 'Burkina Faso' 'Bangladesh'
 'Bulgaria' 'Bahrain' 'Bahamas

KeyboardInterrupt: 